<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Implementing_an_LSTM_for_time_series_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

# Assuming you have your time-series data in sequences and labels tensors
sequences = torch.randn(1000, 10, 1)  # Example data: 1000 sequences, each of length 10 and feature size 1
labels = torch.randn(1000, 1)         # Example labels

# Create dataset and DataLoader
dataset = TimeSeriesDataset(sequences, labels)
trainloader = DataLoader(dataset, batch_size=32, shuffle=True)

# LSTM model definition
class LSTMTimeSeries(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMTimeSeries, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Take the last time step's output
        return out

# Example usage
model = LSTMTimeSeries(input_size=1, hidden_size=50, num_layers=2, output_size=1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(100):
    for sequences, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()